In [1]:

####################################################################################################
####################################################################################################

import sys
import os.path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

import keras
import keras.utils.vis_utils
from keras import backend as K
from keras import layers
from keras.layers import Dense
from keras.models import load_model
from keras.models import Sequential
from keras.utils import plot_model

from helpers import *

sys.path.append(os.path.abspath("../"))
from pax_utils import s1s2_utils



Using TensorFlow backend.


In [7]:

####################################################################################################
####################################################################################################

file_all       = "../pax_merge/merged/merged_all_1803031449.pkl"

df_all         = pd.read_pickle(file_all)
df_all         = df_all[df_all['intr_count'] == 1].reset_index(drop=True)
df_train_truth = df_all[:][['x', 'y', 's2_electrons', 'intr_s2_electrons']]
df_train_input = df_all[:][s1s2_utils.getS2integralsDataFrameColumns()]


####################################################################################################
####################################################################################################

checkTrainingData(df_train_input, df_train_truth)




Input shape: (17982, 127)
Truth shape: (17982, 4)

Input DataFrame: 


,s2_area_000,s2_area_001,s2_area_002,s2_area_003,s2_area_004,s2_area_005,s2_area_006,s2_area_007,s2_area_008,s2_area_009,...,s2_area_117,s2_area_118,s2_area_119,s2_area_120,s2_area_121,s2_area_122,s2_area_123,s2_area_124,s2_area_125,s2_area_126
0,1.715228,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.293108,...,6.564991,0.0,0.400128,2.742959,3.460256,8.159419,38.424745,15.890681,1.354377,10.354574
1,0.912039,0.0,0.0,1.545933,0.000000,0.000000,2.568431,0.000000,2.604376,2.010018,...,3.208600,0.0,0.000000,5.596787,26.363896,126.844246,55.087875,8.130994,8.979807,13.682349
2,0.000000,0.0,0.0,0.000000,2.104002,0.000000,1.731047,2.784006,0.000000,5.977608,...,3.487908,0.0,1.534094,0.000000,1.919893,4.920848,0.505347,2.435258,0.506562,4.203870
3,6.140486,0.0,0.0,9.062252,5.621141,1.789005,1.802042,0.000000,0.869444,0.000000,...,0.000000,0.0,0.918565,0.619911,0.000000,1.230013,0.788772,1.179353,2.046051,0.000000
4,4.333349,0.0,0.0,0.000000,1.313044,0.000000,0.613111,0.000000,1.061757,3.775456,...,12.939582,0.0,3.896350,2.259100,3.023539,0.855252,7.456719,1.257868,4.551362,5.000204



Truth DataFrame:


,x,y,s2_electrons,intr_s2_electrons
0,2.626434,-17.960824,79,98.696105
1,10.907536,-6.312277,75,93.844360
2,24.631697,-12.763601,73,86.820609
3,2.726006,37.749081,70,99.468229
4,-18.078293,-23.212466,109,124.397948


In [8]:

####################################################################################################
####################################################################################################

folder         = "models" + "/"    
model_name     = "model_barge_xyS2_elu"
model_name_h5  = folder + model_name + ".h5"
model_name_png = folder + model_name + ".png"



In [9]:

####################################################################################################
####################################################################################################

model       = load_model(model_name_h5)
arr_xy_pred = model.predict(df_train_input)

print("\nLoaded Model: " + model_name_h5)
print("Predicted\n")




Loaded Model: models/model_barge_xyS2_elu.h5
Predicted



In [10]:

####################################################################################################
####################################################################################################

arr_x_true = df_train_truth[:]['x'].as_matrix()
arr_y_true = df_train_truth[:]['y'].as_matrix()
arr_n_true = df_train_truth[:]['s2_electrons'].as_matrix()
arr_n_reco = df_all[:]['intr_s2_electrons'].as_matrix()

arr_x_pred = arr_xy_pred[:, 0]
arr_y_pred = arr_xy_pred[:, 1]
arr_n_pred = arr_xy_pred[:, 2]



In [12]:

####################################################################################################
####################################################################################################

file_out         = "./predictions/xyS2_S2integrals.pkl"

cols_out = ['event_number', 'x', 'y', 'intr_x', 'intr_y', 'intr_x_nn', 'intr_y_nn', 's2_electrons', 'intr_s2_electrons']

df_out                      = df_all[:][cols_out]
df_out['x_pred']            = arr_x_pred
df_out['y_pred']            = arr_y_pred
df_out['s2_electrons_pred'] = arr_n_pred


df_out.to_pickle(file_out)


####################################################################################################
####################################################################################################

df_test = pd.read_pickle(file_out)

display(df_test[0:5][:])
print()



,event_number,x,y,intr_x,intr_y,intr_x_nn,intr_y_nn,s2_electrons,intr_s2_electrons,x_pred,y_pred,s2_electrons_pred
0,1,2.626434,-17.960824,3.117168,-16.583333,3.270800,-16.607226,79,98.696105,2.082255,-20.254887,89.755188
1,3,10.907536,-6.312277,9.850251,-5.610902,9.921544,-5.569570,75,93.844360,11.073241,-5.387373,87.329346
2,5,24.631697,-12.763601,24.563283,-12.343985,24.643193,-12.371619,73,86.820609,26.674236,-11.689880,102.864540
3,6,2.726006,37.749081,2.119674,34.039474,2.042468,33.434272,70,99.468229,4.525731,38.801102,72.480247
4,8,-18.078293,-23.212466,-16.583333,-20.822682,-16.423884,-21.070236,109,124.397948,-21.186893,-22.057798,110.169212
